In [5]:
import pandas as pd
import numpy as np

In [6]:
import os
for dirname, _, filenames in os.walk('/ashiq/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.stem.snowball import SnowballStemmer
import regex as re
from nltk.tokenize import sent_tokenize
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords

In [6]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [8]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bachu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bachu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bachu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
df_fake = pd.read_csv(r"C:\Users\bachu\Downloads\Intern\archive\News _dataset\Fake.csv")
df_true = pd.read_csv(r"C:\Users\bachu\Downloads\Intern\archive\News _dataset\True.csv")

In [10]:
df_fake

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"


In [12]:
df_true

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"


In [13]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [14]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [15]:
df_true['status'] = 0
df_fake['status'] = 1

In [16]:
#Used to merge & remove unwanted columns

df = pd.concat([df_true,df_fake])
df.drop(['subject','text','date'],axis=1,inplace=True)

In [17]:
random_indexes = np.random.randint(0,len(df),len(df))
df = df.iloc[random_indexes].reset_index(drop=True)

In [18]:
#Text analysis takes place

pd.set_option('display.max_colwidth', 500)
random = np.random.randint(0,len(df),20)
df.iloc[random]

,title,status
26766,WATCH: SEAN HANNITY Offers To Step In After He Sees Heartbreaking Viral Video Of Young Boy Crying Over Being Bullied For His Appearance,1
29243,ERIC HOLDER Encourages DOJ To Keep Attacking Trump…STUNNING List Of Holder Scandals Reveals Why Trump Needs To Drain Obama’s Corrupt DOJ Swamp,1
31019,TRUMP PROMISED TO PUT AMERICANS FIRST AND D.C. TREMBLED: “Their victories have not been your victories; their triumphs have not been your triumphs” [VIDEO],1
29535,Catalonia refuses to send weekly accounts to Madrid before referendum,0
35399,Xi and Trump discuss sanctions pressure on North Korea: White House,0
23042,Emirates stops flying to Tunisia in row over ban on Tunisian women,0
1419,NON-PROFIT VIOLENT Berkeley BAMN Leader Surprised When Tables Are Turned On Him During Interview [VIDEO],1
10258,HUCKABEE NAILS IT: Says Obama’s Radical List Of Invited Guests To Meet Pope Proves His Fake Christianity,1
35979,Trump dismisses Buffett's rebuke over his business judgment,0
21035,WOW! WHAT HAPPENED When Somebody Asked Beyonce’s Racist Sister To Sit Down At A Concert?,1


In [19]:
df.isnull().sum()

title     0
status    0
dtype: int64

In [20]:
#Calculating longest sentence length

def longest_sentence_length(text):
  return len(text.split())

df['maximum_length'] = df['title'].apply(lambda x : longest_sentence_length(x))
print('longest sentence having length -')
max_length = max(df['maximum_length'].values)
print(max_length)

longest sentence having length -
42


In [21]:
#Used to remove non-alphanumeric characters and digits, enhancing text cleanliness for subsequent analysis

text_cleaning = "\b0\S*|\b[^A-Za-z0-9]+"

def preprocess_filter(text, stem=False):
  text = re.sub(text_cleaning, " ",str(text.lower()).strip())
  tokens = []
  for token in text.split():
    if token not in stop_words:
      if stem:
        stemmer = SnowballStemmer(language='english')
        token = stemmer.stem(token)
      tokens.append(token)
  return " ".join(tokens)

In [22]:
def one_hot_encoded(text,vocab_size=5000,max_length = 40):
    hot_encoded = one_hot(text,vocab_size)
    return hot_encoded

In [23]:
def word_embedding(text):
    preprocessed_text=preprocess_filter(text)
    return one_hot_encoded(preprocessed_text)

In [24]:
# Creating a NN(Neural Network) Model

embedded_features = 40
model = Sequential()
model.add(Embedding(5000,embedded_features,input_length = max_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy',optimizer= 'adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [25]:
one_hot_encoded_title =df['title'].apply(lambda x : word_embedding(x)).values

In [26]:
padded_encoded_title = pad_sequences(one_hot_encoded_title,maxlen=max_length,padding = 'pre')

In [27]:
# Splitting 
X = padded_encoded_title
y = df['status'].values
y = np.array(y)

# shapes

print(X.shape)
print(y.shape)

(44898, 42)
(44898,)


In [28]:
print('X shape {}'.format(X.shape))
print('y shape {}'.format(y.shape))

X shape (44898, 42)
y shape (44898,)


In [30]:
# Splitting into training, testing

X_train,X_test,y_train,y_test = train_test_split(X,y, random_state = 42)

print('X train shape {}'.format(X_train.shape))
print('X test shape {}'.format(X_test.shape))
print('y train shape {}'.format(y_train.shape))
print('y test shape {}'.format(y_test.shape))

X train shape (33673, 42)
X test shape (11225, 42)
y train shape (33673,)
y test shape (11225,)


In [31]:
# Model training

model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=15,batch_size=64)

Epoch 1/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - accuracy: 0.8532 - loss: 0.3229 - val_accuracy: 0.9537 - val_loss: 0.1176
Epoch 2/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 13s 25ms/step - accuracy: 0.9743 - loss: 0.0756 - val_accuracy: 0.9613 - val_loss: 0.1052
Epoch 3/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 14s 26ms/step - accuracy: 0.9860 - loss: 0.0409 - val_accuracy: 0.9674 - val_loss: 0.1085
Epoch 4/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 14s 27ms/step - accuracy: 0.9930 - loss: 0.0231 - val_accuracy: 0.9690 - val_loss: 0.1160
Epoch 5/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 13s 25ms/step - accuracy: 0.9964 - loss: 0.0133 - val_accuracy: 0.9654 - val_loss: 0.1346
Epoch 6/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 13s 25ms/step - accuracy: 0.9962 - loss: 0.0108 - val_accuracy: 0.9674 - val_loss: 0.1499
Epoch 7/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 13s 25ms/step - accuracy: 0.9974 - loss: 0.0070 - val_accuracy: 0.9676 - val_loss: 0.1719
Epoch 8/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 14s 26ms/step - accuracy: 0.9983 - loss: 0.0060 - 

In [32]:
#setting threshold value for evaluation

def best_threshold_value(thresholds:list,X_test):
    accuracies = []
    for thresh in thresholds:
        ypred =model.predict(X_test)
        ypred = np.where(ypred> thresh,1,0)
        accuracies.append(accuracy_score(y_test,ypred))
    return pd.DataFrame({
        'Threshold': thresholds,
        'Accuracy' : accuracies
    })

In [33]:
best_threshold_value([0.4,0.5,0.6,0.7,0.8,0.9], X_test)

351/351 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step
351/351 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step
351/351 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step
351/351 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step
351/351 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step
351/351 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step


,Threshold,Accuracy
0,0.4,0.968107
1,0.5,0.968196
2,0.6,0.968552
3,0.7,0.968820
4,0.8,0.969800
5,0.9,0.969532


In [34]:
y_pred = model.predict(X_test)
y_pred = np.where(y_pred >0.4, 1, 0)

351/351 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step


In [35]:
# Confusion matrix

print('Confusion matrix')
print(confusion_matrix(y_pred,y_test))
print('----------------')
print('Classification report')
print(classification_report(y_pred,y_test))

Confusion matrix
[[5160  170]
 [ 188 5707]]
----------------
Classification report
              precision    recall  f1-score   support

           0       0.96      0.97      0.97      5330
           1       0.97      0.97      0.97      5895

    accuracy                           0.97     11225
   macro avg       0.97      0.97      0.97     11225
weighted avg       0.97      0.97      0.97     11225



In [40]:
#input generator for prediction

def prediction_input_processing(text):
    encoded = word_embedding(text)
    padded_encoded_title = pad_sequences([encoded],maxlen=max_length,padding = 'pre')
    output = model.predict(padded_encoded_title)
    output = np.where(0.4>output,1,0)
    if output[0][0] == 1:
        return 'Yes, this News is fake'
    return 'No, It is not fake'   

In [41]:
# prediction no.1

prediction_input_processing('Americans are more concerned over Indians fake open source contribution')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step


'Yes, this News is fake'

In [42]:
# prediction no.2

news = 'Trump Just Sent Michelle Obama a Bill She will Never Be able to pay in her lifetime'
prediction_input_processing(news)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


'No, It is not fake'